Analyse socio-économique et délinquance, 2012-2016

In [2]:
import pandas as pd

data = pd.read_csv("full_dataset_V2.csv")

C:\Users\thiba\AppData\Local\Temp\ipykernel_7008\3138384080.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("full_dataset_V2.csv")


In [3]:
data_2012 = data[data['annee']==2012]

data_2012 = data_2012[~(data_2012['code_dep'] == 'France_Métro')]

In [4]:
data_2012.head()

,Date,code_dep,Homicides,Vols,Cambriolages à domicile ou violaiton de domicile,Crimes ou délits sexuelles,Infractions liées aux stupéfiants,Dégradations ou destructions de biens volontaires,Violences,Falsifications et contrefaçons,...,Taux de pauvreté-Ensemble,Bac_g_réussite_%,Bac_t_réussite_%,Bac_p_réussite_%,Bac_g_%,Bac_t_%,Bac_p_%,non_peu_diplômés_non_scolarisée_<15ans,Superficie\n(km2),effectif_total_police_national
360,2012-12-31,01,0,765,251,28,125,98,136,222,...,10.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5762.4,297
361,2012-11-30,01,1,834,250,16,134,133,155,204,...,10.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5762.4,297
362,2012-10-31,01,2,865,157,27,140,127,187,213,...,10.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5762.4,297
363,2012-09-30,01,0,754,147,20,112,123,154,161,...,10.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5762.4,297
364,2012-08-31,01,0,858,120,20,102,109,145,166,...,10.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5762.4,297


In [5]:
data_2012.dtypes

Date                                                  object
code_dep                                              object
Homicides                                              int64
Vols                                                   int64
Cambriolages à domicile ou violaiton de domicile       int64
Crimes ou délits sexuelles                             int64
Infractions liées aux stupéfiants                      int64
Dégradations ou destructions de biens volontaires      int64
Violences                                              int64
Falsifications et contrefaçons                         int64
Délits Économiques et Financiers                       int64
Irégularités de Main-d'oeuvre                          int64
annee                                                  int64
Population                                           float64
Chomage                                                int64
Taux de pauvreté-moins de 30                         float64
Taux de pauvreté-30 à 39

In [7]:
data_2012['Population'] = pd.to_numeric(data_2012['Population'], errors='coerce')

In [9]:
#Dégradations ou destructions de biens volontaires

#Population

#effectif_total_police_national

taux_effectif_total_police_national = data_2012['effectif_total_police_national'] / data_2012['Population']

taux_degradation_ou_destruction = data_2012['Dégradations ou destructions de biens volontaires'] / data_2012['Population']

TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


# Union des deux taux


#Calcul du taux de corrélation entre ces deux taux 
.corr(method="pearson")

# Visualize the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(, annot=True, cmap='coolwarm')
plt.title('Matrice de corrélation entre dégradations et effectifs de police ')
plt.show()